In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
# Create a new instance of the Firefox driver

driver = webdriver.Firefox()
url = 'https://www.jeyamohan.in/category/%E0%AE%B5%E0%AE%BE%E0%AE%9A%E0%AE%95%E0%AE%B0%E0%AF%8D%E0%AE%95%E0%AE%B3%E0%AF%8D/%E0%AE%95%E0%AF%87%E0%AE%B3%E0%AF%8D%E0%AE%B5%E0%AE%BF-%E0%AE%AA%E0%AE%A4%E0%AE%BF%E0%AE%B2%E0%AF%8D/'
# Navigate to the web page
driver.get(url)

def class_select():
    # Find the link based on class search
    link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "td_ajax_load_more"))
    )
    return link

def css_select():
    link = driver.find_element_by_css_selector("a:contains(' மேலும் காண ' or  'மேலும் காண' )")
    return link
z = 0
while True:
    link = class_select()
    if link == '':
        link = css_select()
    if link =='':
        break
    driver.execute_script("arguments[0].click();", link)
    driver.execute_script("arguments[0].scrollIntoView();", link)
    time.sleep(2)
    z+=1
    
links = driver.execute_script("return Array.from(document.querySelectorAll('a')).map(a => a.href);")
qa_links = []
for i in links:
    if re.search(r"https://www\.jeyamohan\.in/\d+/", i):
        qa_links.append(i)
print(len(qa_links))


KeyboardInterrupt: 

In [28]:
qa_links = list(set(qa_links))

In [30]:
import json
 
with open("my_list.json", "w") as f:
    json.dump(qa_links, f, indent=4)


In [80]:
import json
path = 'jayamohan_kelvi_pathil_links.json'
with open(path, "r") as f:
    links = json.load(f)
len(links)

1733

In [82]:
import requests
from bs4 import BeautifulSoup
import string
from tqdm import tqdm
# Make a request to the webpage
ans_ls = []
for i in tqdm(links):
    response = requests.get(i)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the element with the 'td-post-content' class
    post_content = soup.find(class_='td-post-content')
    txt = post_content.get_text()    
    ls = txt.split('ஜெ')    
    for i in ls:
        my_dict = {}
        if len(i)>20:
            translator = str.maketrans('', '', string.punctuation)
            i = i.replace('யமோகன்','').replace('\n',' ').replace('\xa0',' ').replace("'",'').replace('”','').replace('–','').replace('‘','')
            i = i.translate(translator)
            my_dict['question'] = i.split('அன்புள்ள')[0]
            my_dict['answer'] = i.split('அன்புள்ள')[-1]
            if len(my_dict['question']) >25:
                if len(my_dict['answer']) >25:
                    if my_dict['question']!=my_dict['answer']:
                        ans_ls.append(my_dict)        
with open('jayamohan_qna.json', 'w',encoding='utf-8') as f:
    json.dump(ans_ls, f, ensure_ascii=False, indent=4)

100%|█████████████████████████████████████| 1733/1733 [1:08:35<00:00,  2.37s/it]
